# import data

In [15]:
import os 
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [2]:
data_path = '/Users/bokie/code/python/CMS/可转债套利/cms/stat_arb/stk_data'
df_price_dic = {}
df_return_dic = {}
industry = os.listdir(data_path+'/price')

for tmp_industry in industry:
    tmp_price_path = data_path + '/price/' + tmp_industry
    tmp_return_path = data_path + '/return/' + tmp_industry
    tmp_df_price = pd.read_pickle(tmp_price_path)
    tmp_df_return = pd.read_pickle(tmp_return_path)
    tmp_key = tmp_industry.split('.')[0]
    df_price_dic[tmp_key] = tmp_df_price
    df_return_dic[tmp_key] = tmp_df_return

In [3]:
corr_dic = {}
for k,v in df_return_dic.items():
    corr_mat = v.corr(method='pearson')
    for i in range(len(corr_mat)):
        corr_mat.iloc[i,i] = np.nan
    corr_dic[k] = corr_mat

# pair correlation

In [7]:
corr_pair_dic = {}
for tmp_industry in corr_dic.keys():
    tmp_corr_max = corr_dic[tmp_industry].max()
    tmp_df_corr = corr_dic[tmp_industry]

    tmp_corr_max,type(tmp_corr_max),tmp_corr_max.index.tolist()
    industry_corr_pair_dic = {}

    main_code_list = tmp_corr_max.index.tolist()
    for main_code in main_code_list:
        tmp_line = tmp_df_corr.loc[main_code]
        tmp_line = pd.DataFrame(tmp_line)
        for index,row in tmp_line.iterrows():
            if row[main_code] == tmp_corr_max.loc[main_code]:
                # print(index)
                sec_code = index
                tmp_corr = tmp_corr_max.loc[main_code]
        # print(f'\'{main_code}\',\'{sec_code}\'','@',tmp_corr)
        tmp_key = main_code+'@'+sec_code
        tmp_key_reverse = sec_code+'@'+main_code
        if (tmp_key not in industry_corr_pair_dic) and (tmp_key_reverse not in industry_corr_pair_dic):
            industry_corr_pair_dic[tmp_key] = tmp_corr
        
    corr_pair_dic[tmp_industry] = sorted(industry_corr_pair_dic.items(), key=lambda x:x[1],reverse=True)


In [26]:
df_corr_pair = pd.DataFrame(columns=['industry','corr'])
for k,v in corr_pair_dic.items():
    for pair_corr in v:
        tmp_pair = pair_corr[0]
        tmp_corr = pair_corr[1]
        if tmp_corr>0:
            # print(k,tmp_pair,tmp_corr)
            df_corr_pair.loc[tmp_pair] = [k,tmp_corr]
            # print("%-10s\t\t%-10s\t\t%-10s"%(k,tmp_pair,tmp_corr))
df_corr_pair = df_corr_pair.reset_index()
df_corr_pair.columns = ['pair','industry','corr']
df_corr_pair

,pair,industry,corr
0,000625.SZ@600104.SH,汽车,0.389785
1,601127.SH@000625.SZ,汽车,0.377956
2,002488.SZ@600699.SH,汽车,0.288415
3,603306.SH@600699.SH,汽车,0.259472
4,000088.SZ@600798.SH,交通运输,0.521987
...,...,...,...
147,603868.SH@603355.SH,家用电器,0.206898
148,601952.SH@002505.SZ,农林牧渔,0.333770
149,002556.SZ@601952.SH,农林牧渔,0.263925
150,002714.SZ@002505.SZ,农林牧渔,0.234709


# pair cointegration test

In [28]:
df_return_dic['煤炭'][['600395.SH','002128.SZ']]

,600395.SH,002128.SZ
2020-10-29 15:00:00,0.000000,0.000000
2020-10-30 09:30:00,0.003339,0.002020
2020-10-30 09:31:00,0.000000,0.002020
2020-10-30 09:32:00,0.000000,-0.001010
2020-10-30 09:33:00,0.001669,-0.001010
...,...,...
2022-10-28 14:56:00,-0.035763,-0.024295
2022-10-28 14:57:00,-0.034388,-0.025078
2022-10-28 14:58:00,-0.034388,-0.025078
2022-10-28 14:59:00,-0.034388,-0.025078


In [29]:

def cointegration_test(y, x):
    ols_result = sm.OLS(y, x).fit()
    return adfuller(ols_result.resid)
pair_return = df_return_dic['煤炭'] 
pair_code = ['600395.SH','002128.SZ']
x_code = pair_code[0]
y_code = pair_code[1]   
x = pair_return[x_code].tolist()
y = pair_return[y_code].tolist()

cointegration_test(y, x),cointegration_test(x, y)

((-19.258415293098768,
  0.0,
  2,
  117368,
  {'1%': -3.4304057174274587,
   '5%': -2.8615646262701717,
   '10%': -2.566783107694885},
  -1057463.4108017215),
 (-18.982326608505577,
  0.0,
  30,
  117340,
  {'1%': -3.4304057307231983,
   '5%': -2.8615646321466346,
   '10%': -2.566783110822729},
  -1045208.311267216))